In [ ]:
import numpy as np
import pandas as pd

import pitched_recommend

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, AffinityPropagation
from sklearn import cluster, metrics

import matplotlib.pyplot as plt
import seaborn as sns

import pickle

import timeit

sns.set(style="darkgrid", palette="Set3")

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
with open('/Users/meshchd/Downloads/saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
df = pd.DataFrame(data=model.playlist_vecs, index=model.playlists, 
                  columns=['factor'+str(x) for x in range(1,model.playlist_vecs.shape[1]+1)])
df.head()

In [ ]:
df.shape

In [ ]:
df_small = df.iloc[:200000,:]
df_small.shape

In [ ]:
df_chris = pd.DataFrame(data=model.isrc_vecs, index=model.isrcs, 
                        columns=['factor'+str(x) for x in range(1,model.playlist_vecs.shape[1]+1)])
df_chris.head()

In [ ]:
df_chris = df_chris.reset_index()

In [ ]:
df_chris.to_gbq(destination_table='adhoc.isrc_factors_for_chris', project_id='umg-comm-tech-dev', if_exists='replace')

In [ ]:
df_chris.to_feather('/Users/meshchd/Downloads/isrc_factors_for_chris.feather')

### DBSCAN

In [ ]:
# Function to run clutering algorithms and review their output

def dbscan_tuning(df,eps_list,min_samples_list):
    
    
    start = timeit.timeit()
    #print("Start time: ", start)
    df_res = pd.DataFrame(columns=['eps','min_sample','n_clusters','unlabelled_prop','silhouette_coeff'],
                         index = range(len(eps_list)*len(min_samples_list)))
    c=-1
    
    for i,eps_i in enumerate(eps_list):
        for j, min_j in enumerate(min_samples_list):
            
            print("Running iteration for eps ", eps_i)
            
            c=c+1
            
            dbscn = DBSCAN(eps=eps_i, min_samples=min_j)
            dbscan=dbscn.fit(df)
            labels = dbscan.labels_ 
            n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
            
            #print("epsilon, min_samples: ", eps_i, min_j)
            #print( "clusters: ", n_clusters_)
            #print("unlabelled proportion: ", float(list(labels).count(-1))/labels.shape[0])
            #print("silhouette coefficient: ", metrics.silhouette_score(df, labels))
            #print()
            
            df_res['eps'][c]=eps_i
            df_res['min_sample'][c]=min_j
            df_res['n_clusters'][c]=n_clusters_
            df_res['unlabelled_prop'][c]=float(list(labels).count(-1))/labels.shape[0]
            df_res['silhouette_coeff'][c]=metrics.silhouette_score(df, labels)
            
    end = timeit.timeit()
    print("elapsed time: ", end-start)
    
    return df_res

In [ ]:
# Range of clustering parameters

eps_list=[50]
min_samples_list=[3]

In [ ]:
df_res = dbscan_tuning(df,eps_list,min_samples_list)

In [ ]:
df_res.head(10)

In [ ]:
ax = sns.scatterplot(x="n_clusters", y="silhouette_coeff", data=df_res)

### Affinity Propagation

In [ ]:
af = AffinityPropagation(verbose=True).fit(df_small)

In [ ]:
print('Estimated number of clusters: %d' % n_clusters_)
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels, metric='sqeuclidean'))

In [ ]:
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_